In [2]:
import networkx as nx
from collections import defaultdict
import heapq as hp
import folium
from folium import plugins

In [4]:
with open(r'USA-road-d.CAL.co','r') as f1:
    for line in f1:
        if line[0] == 'v':
            n,la,lo = list(map(int, line[2:].split()))
            G.add_node(n,latitude = la/1000000,longitude = lo/1000000)

In [6]:
adj = defaultdict(set)
with open(r'USA-road-d.CAL.gr','r') as f:
    for line in f:
        if line[0] == 'a':
            n1,n2, d =  list(map(int, line[2:].split()))
            G.add_edge(n1,n2,distance = d,weight = 1)
            adj[n1].add(n2)
            adj[n2].add(n1)

In [16]:
def dijkstra(v,end,p,graph = G,adjacent = adj):
    # v is the start node
    # end is the destination node
    # Also here as in the functionality 1 I use djikstra with small changes here instead of calculating 
    # the neighbors of the starting node I calculate the minimum path between the starting node and the arrival node.
    # I create a dictionary of the visited ones but besides the weight I save the predecessor
    # that then it will be useful to me to reconstruct the path
    visited = {v: (None, 0)}
    F = []
    hp.heapify(F)
    current_node = v
    #the core part of the code is the same with the exception that as the output of the while 
    #I have reached the destination node
    while current_node != end:
        adjacentes = adjacent[current_node]
        weight_to_current_node = visited[current_node][1]
        for node in adjacentes:
            weight = graph[current_node][node][p] + weight_to_current_node
            if node not in visited:
                visited[node] = (current_node, weight)
                hp.heappush(F,(weight,node))
            else:
                current_shortest_weight = visited[node][1]
                if current_shortest_weight > weight:
                    visited[node] = (current_node, weight)
                    hp.heappush(F,(weight,node))
        #also here I check if the border is empty, if it's empty and we haven't left yet while it means that 
        #I checked all the nodes without having reached the destination node 
        #so I can say that there is no path between the starting node and the one of destination
        if not F:
            return "Route Not Possible"
        current_node = hp.heappop(F)[1]
    # Work backwards between the destinations visited up to the node that has no parent set to None, then up to the
    #first node. I start from the current node because after the while the current node is the destination node
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = visited[current_node][0]
        current_node = next_node
     # having added the nodes in the list from the last one obviously before returning it I reverse it
    path.reverse()
    return path

In [10]:
with open(r'USA-road-t.CAL.gr','r') as f2:
    for line in f2:
        if line[0] == 'a':
            n1,n2, t =  list(map(int, line[2:].split()))
            G.add_edge(n1,n2,time = t)

## Functionality 2

In [90]:
def function2(nodes, d):
    #Initialising two lists that are built simultaneously, sh_paths to keep the minimal (Dijkstra) paths 
    #(as lists of nodes) between nodes given in input, lengths to keep the distance of each minmal path 
    #at the same index of the paths in sh_paths
    sh_paths = []
    hp.heapify(sh_paths)
    newset = nodes.copy()
    for i in nodes:
        #newset is necessary in order to take only once each minimal path, so we don't take 
        #the same nodes as starting and ending point
        newset.remove(i)
        for j in newset:
            path = dijkstra(graph=G, v=i, end= j, p = d)
            if not type(path) == str:
                length = sum([G[path[k-1]][path[k]][d] for k in range(1, len(path))])
                hp.heappush(sh_paths, (length, path))
    #The algorithm starts taking the minimum path between all paths that connect all nodes from the input, and storing
    #a set of visited nodes starting with the first two, and a set of arcs that form the paths
    visited = set()
    short = sh_paths.copy()
    p = hp.heappop(short)[1]
    visited.add(p[0])
    visited.add(p[-1])
    out = set([(p[i-1], p[i]) for i in range(1,len(p))])
    while visited != nodes:
        #the loop goes on until all nodes given in input are visited, and takes always the minimum of the 
        #remaining paths that connects only one of the visited nodes to one of the not visited; this way, we can
        #be sure that we are connecting these minimal paths to the same network
        p = hp.heappop(short)[1]
        if len({p[0], p[-1]}.intersection(visited)) == 1:
            out = out.union(set([(p[i-1], p[i]) for i in range(1,len(p))]))
            visited.add(p[0])
            visited.add(p[-1])
            short = sh_paths.copy()
            
        #this condition is used to to check if the nodes passed in input aren't connected to each other. If that's the
        #case, so we have checked all of the paths but we didn't mange to visit all the nodes, than we should try to
        #reapply recursively the same function on the subgraph of the nodes not yet visited, and see if we manage to 
        #connect any other node that has no connection to the original network
        if len(short) == 0 and len(visited)>1:
            out = out.union(function2(nodes-visited, d))
    return out


In [86]:
def fun2_visual(subset, w = 'weight'):
    locator = G.nodes[list(subset)[0]]
    ourmap = folium.Map(location=[locator['longitude'], locator['latitude']], zoom_start=10)
    for i in subset:
        coor = G.nodes[i]
        folium.Marker(location=[(coor['longitude']),(coor['latitude'])], popup = str(i),
                          icon=plugins.BeautifyIcon(icon_shape = 'circle-dot'
                            ,border_color = 'Blue')).add_to(ourmap)
    sh_paths = []
    lengths = []
    newset = subset.copy()
    for i in subset:
        newset.remove(i)
        for j in newset:
            path = dijkstra(graph=G, v=i, end= j, p = w)
            length = sum([G[path[k-1]][path[k]][w] for k in range(1, len(path))])
            lengths.append(length)
            sh_paths.append(path)
    for p in sh_paths:
        points = []
        for node in p:
            coor = G.nodes[node]
            points.append((coor['longitude'], coor['latitude']))
        folium.vector_layers.PolyLine(points, color = 'Yellow').add_to(ourmap)
    for edge in function2(subset, w):
        points = []
        for node in edge:
            coor = G.nodes[node]
            points.append((coor['longitude'], coor['latitude']))
        if w == 'distance':
            folium.vector_layers.PolyLine(points, color = 'Red').add_to(ourmap)            
        elif w == 'time':
            folium.vector_layers.PolyLine(points, color = 'Green').add_to(ourmap)
        elif w == 'weight':
            folium.vector_layers.PolyLine(points, color = 'Purple').add_to(ourmap)
    return ourmap


In [91]:
fun2_visual({1, 23,122,1443211}, 'time')

## Functionality 4 (not Ready)

In [101]:
st_node = 8272
subset_1 = {3,7,223,8,134}

In [102]:
sh_paths = []
all_nodes = []
lengths = []
newset_1 = subset_1.copy()

for j in newset_1:
    path = nx.shortest_path(G, source = st_node, target = j, weight = 'distance')
    length = sum([G[path[i-1]][path[i]]['distance'] for i in range(1, len(path))])
    lengths.append(length)
    sh_paths.append(path)
    all_nodes += path

In [103]:
lengths

[1442254, 1694146, 1703497, 1597791, 1778174]

In [104]:
newmap = folium.Map(location=[G.nodes[st_node]['longitude'], G.nodes[st_node]['latitude']], zoom_start=10)

In [105]:
newmap

In [106]:
coor = G.nodes[st_node]
folium.Marker(location=[(coor['longitude']),(coor['latitude'])], popup = str(i),
                      icon=plugins.BeautifyIcon(icon_shape = 'circle-dot'
                        ,border_color = 'Red')).add_to(newmap)

In [107]:
for i in subset_1:
    coor = G.nodes[i]
    folium.Marker(location=[(coor['longitude']),(coor['latitude'])], popup = str(i),
                      icon=plugins.BeautifyIcon(icon_shape = 'circle-dot'
                        ,border_color = 'Green')).add_to(newmap)

In [108]:
for p in sh_paths:
    points = []
    for node in p:
        coor = G.nodes[node]
        points.append((coor['longitude'], coor['latitude']))
    folium.vector_layers.PolyLine(points).add_to(newmap)

In [109]:
newmap